In [0]:
!nvidia-smi


Thu May  7 16:19:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
from keras.layers import *
from keras.models import *
from keras.optimizers import SGD
import warnings
warnings.filterwarnings('ignore')


def conv_block(X, kernel_nb, with_short_cut=False):
    x = Conv2D(kernel_nb, (3, 3), padding='same', strides=(1, 1))(X)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x = Conv2D(kernel_nb, (3, 3), padding='same', strides=(1, 1))(x)
    x = BatchNormalization(axis=-1)(x)
    if with_short_cut:
        shortcut = Conv2D(kernel_nb, (3, 3), padding='same', strides=(1, 1))(X)
        shortcut = BatchNormalization(axis=-1)(shortcut)
        x = add([x, shortcut])
    else:
        x = add([x, X])
    x = Activation('relu')(x)
    return x


input = Input(shape=(28, 28, 3))
# layer1
l11 = conv_block(input, 128, with_short_cut=True)
l12 = conv_block(l11, 128)
pool1 = MaxPool2D(pool_size=(2, 2), strides=(2, 2))(l12)

# layer2
l21 = conv_block(pool1, 256, with_short_cut=True)
l22 = conv_block(l21, 256)
pool2 = MaxPool2D(pool_size=(2, 2), strides=(2, 2))(l22)

# layer3
l31 = conv_block(l22, 256, with_short_cut=True)
l32 = conv_block(l31, 256)
avePool = AveragePooling2D(pool_size=(2, 2), strides=(2, 2))(l32)

flatten = Flatten()(avePool)
dense = Dense(32)(flatten)
batchnormalization = BatchNormalization(axis=1)(dense)
act = Activation('relu')(batchnormalization)
output = Dense(7, activation='softmax')(act)
ResModel = Model(input, output)

sgd = SGD(learning_rate=0.05, decay=0.9, momentum=0.9, nesterov=True)
ResModel.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
ResModel.summary()


import pandas as pd
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
df = pd.read_csv('hmnist_28_28_RGB.csv')
train, test = train_test_split(df, test_size=0.1)
train_x = train.drop('label', axis=1).values.reshape([-1, 28, 28, 3])
train_y = to_categorical(train['label'].values)
test_x = test.drop('label', axis=1).values.reshape([-1, 28, 28, 3])
test_y = to_categorical(test['label'].values)

ResModel.fit(train_x, train_y, epochs=20, validation_split=0.2, batch_size=100, verbose=1)

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 28, 28, 3)    0                                            
__________________________________________________________________________________________________
conv2d_46 (Conv2D)              (None, 28, 28, 128)  3584        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_49 (BatchNo (None, 28, 28, 128)  512         conv2d_46[0][0]                  
__________________________________________________________________________________________________
activation_40 (Activation)      (None, 28, 28, 128)  0           batch_normalization_49[0][0]     
____________________________________________________________________________________________